# Results analysis: machine learning

In [ ]:
from bokeh.charts import Bar
from bokeh.io import output_notebook, show, output_file, reset_output, gridplot, save
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource,Range1d, LinearAxis
from bokeh.palettes import YlOrRd7, Spectral7, RdBu10, PuBu9, RdYlGn10, OrRd9, YlGn9
from bokeh.models import Span
from datetime import datetime
from pandas.tseries.offsets import *
from scipy import stats

import ast
import numpy as np
import pandas as pd
import os

In [ ]:
import resultsAnalysis_utils as uti

In [ ]:
from resultsAnalysis_dataLoading import *

## ML evaluation

### ML results loading

In [ ]:
mlResultsFilePath = '../results/dae/neuralNetwork/performances_ML.csv'

In [ ]:
mlResults = pd.read_csv(filepath_or_buffer=mlResultsFilePath,
                        sep=';',
                        header=0)

for i, param in enumerate(parametersSetML):
    concatParams = mlResults['concatParams']
    mlResults[param] = concatParams.str.split('_').str[i]
mlResults = mlResults.replace('None', -1)
for col in numericML: mlResults[[col]] = mlResults[[col]].apply(pd.to_numeric)

In [ ]:
# Delete these columns because it's the same for every record
del mlResults['activFirstLayer']
del mlResults['activSecondLayer']
del mlResults['loss_func']
del mlResults['optimizer']
del mlResults['scalingFactor']

### Quick stats

In [ ]:
mlResults['loss'].describe()

In [ ]:
mlResults['smoothness'].describe()

In [ ]:
mlResults

### Model evaluation
- Sort models according to training loss and smoothness.
- Compare the 2 measures.

#### Loss

In [ ]:
sortedLoss = uti.rawSort(results=mlResults, metricToSortBy='loss')

In [ ]:
# sortedLoss[['loss', 'smoothness', 'encoding_dim', 'denoising', 'dropoutProba', 'nb_epoch', 'batch_size']].head(51).to_latex()

Data compression seems to work better: less overfitted solutions

In [ ]:
uti.displayMetricEvolution(data=sortedLoss, metric='loss')

#### Smoothness

In [ ]:
sortedSmoothness = uti.rawSort(results=mlResults, metricToSortBy='smoothness')

In [ ]:
sortedSmoothness.head(5)

Same comment as above according to smoothness criteria

In [ ]:
uti.displayMetricEvolution(data=sortedSmoothness, metric='smoothness')

### Compare loss and smoothness
Let's compare the two metrics and see how configs are ranked according to loss/smoothness. Colored squares stand for a configuration.
- Configs that lie on diagonal mean loss/smoothness are ranked the same
- Configs that lie close to the diagonal mean they are almost ranked the same

In [ ]:
uti.xyComparison(sortedSmoothness['concatParams'], sortedLoss['concatParams'])

## Influence of parameters

See influence of each parameter ceteris paribus.
- encoding_dim
- denoising
- dropoutProba

### Encoding dimension

In [ ]:
grouped = mlResults.groupby(['batch_size', 'nb_epoch', 'denoising', 'dropoutProba'])

In [ ]:
uti.displayParameterEvolution(groupedData=grouped, metrics=['loss', 'smoothness'], parameter='encoding_dim')

Regarding data compression, results are in line with common sense: the more compressed, the higher the error/smoothness.

However, regarding data augmentation, we may infer that globally the network learns something close to identity as long as the dropout proba is lower than 0.3, since no matter the encoding dimension, error and smoothness are roughly the same. We may want to choose a network with higher variance, such as a network with dropout proba >= 0.4. In that case, the error does not decrease necessarly with the encoding dimension as one may expect. May be due to high level of randomness and noise.

### Dropout proba

In [ ]:
filtered = mlResults[mlResults['dropoutProba'] != -1]

In [ ]:
grouped = filtered.groupby(['batch_size', 'nb_epoch', 'denoising', 'encoding_dim'])

In [ ]:
uti.displayParameterEvolution(groupedData=grouped, metrics=['loss', 'smoothness'], parameter='dropoutProba')

Results in line with common sense: the noisier the input, the higher the error

## Residuals

### Plot residuals

Epsilon distribution of training and testing sets

In [ ]:
reset_output()
output_notebook()
output_file('../results/dae/neuralNetwork/resultsAnalysis/residuals.html')

stocks = epsilon_test.columns.values

res_test = epsilon_test.reset_index()
res_train = epsilon_train.reset_index()

grid = []
subGrid = []
for i, stock in enumerate(sorted(stocks)):
    if i % 3 == 0 and i != 0:
        grid.append(subGrid)
        subGrid = []
    p1 = figure(background_fill_color="#E8DDCB",plot_width=500, plot_height=500)
    p1.xaxis.axis_label_text_font_size = "12pt"
    p1.legend.location = "top_left"
    p1.xaxis.axis_label = 'r - r_hat'
    p1.yaxis.visible = None
    hist, edges = np.histogram(res_train[stock], density=True, bins=25)
    p1.quad(top=hist,
            bottom=0,
            left=edges[:-1],
            right=edges[1:],
            fill_color="red",
            fill_alpha=0.5,
            legend='train')
    subGrid.append(p1)
    hist, edges = np.histogram(res_test[stock], density=True, bins=25)
    p1.quad(top=hist,
            bottom=0,
            left=edges[:-1],
            right=edges[1:],
            fill_color="green",
            fill_alpha=0.5,
            legend='test')
p = gridplot(grid)
save(p)

### Test homogeneity

Between test and train residuals

In [ ]:
tests=[]
for stock in epsilon_test.columns.values:
    resTest = stats.ks_2samp(epsilon_test[stock], epsilon_train[stock])
    tests.append([stock,
                  round(resTest.statistic,3),
                  round(resTest.pvalue,3)])

In [ ]:
pd.DataFrame(tests).set_index(0).sort_values(by=2)[-10:].to_latex()

In [ ]:
tests = []
for stock in epsilon_test.columns.values:
    testt = stats.ks_2samp(epsilon_test[stock], epsilon_train[stock])
    if testt.pvalue < 0.05:
        tests.append(1)
        print('%s : %f')%(stock, testt.pvalue)
string = '%f des résidus test/train ne sont statistiquements pas issus de la même distribution'
print(string)%(len(tests) * 1. / len(epsilon_test.columns.values))

Make sure residuals for which H0 has been rejected matches a decent distribution

## Residuals and returns volatility

### Correlation between residuals and volatility
Compute epsilon on training and testing samples

In [ ]:
conf = '10_100_False_None_adadelta_mse_200_10_tanh_linear'

In [ ]:
predictions_test = pd.read_csv(filepath_or_buffer='../results/dae/neuralNetwork/predictions/test_' + conf + '.csv',
                               sep=';',
                               header=0,
                               index_col='Date',
                               parse_dates=True)
predictions_train = pd.read_csv(filepath_or_buffer='../results/dae/neuralNetwork/predictions/train_' + conf + '.csv',
                               sep=';',
                               header=0,
                               index_col='Date',
                               parse_dates=True)

In [ ]:
epsilon_test = test - predictions_test
epsilon_train = train - predictions_train

Compute first sigma_return for training period. Start in 2005 so vol can be estimated over the past year.

In [ ]:
def computeVols(df):
    vols = {}
    for day in df.index.values:
        if pd.to_datetime(str(day)) >= datetime(2005,1,1):
            start = pd.to_datetime(str(day)) - DateOffset(months=12)
            dfRescaled = df[start:day]
            dailyVols = dfRescaled.std()
            vols[day] = dailyVols

    return pd.DataFrame.from_dict(vols, orient='index')

In [ ]:
vols_returns = computeVols(train)

Use absolute values to stay in the same universe as volatility (which is positive)

In [ ]:
epsilon_train_abs = epsilon_train.abs()

In [ ]:
epsilon_train_abs.loc['2005':].corrwith(vols_returns).describe()

### Correlation between standard deviation of residuals and volatily

Compute sigma_epsilon based on the same method as for the volatility above

In [ ]:
vols_epsilon = computeVols(epsilon_train)

In [ ]:
vols_epsilon.corrwith(vols_returns).quantile(0.05)